## Jelajahi Klasterisasi K-Means menggunakan R dan Prinsip Data Tidy

### [**Kuis sebelum pelajaran**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Dalam pelajaran ini, Anda akan belajar cara membuat klaster menggunakan paket Tidymodels dan paket lainnya dalam ekosistem R (kita akan menyebutnya teman 🧑‍🤝‍🧑), serta dataset musik Nigeria yang telah Anda impor sebelumnya. Kita akan membahas dasar-dasar K-Means untuk Klasterisasi. Ingatlah bahwa, seperti yang Anda pelajari di pelajaran sebelumnya, ada banyak cara untuk bekerja dengan klaster, dan metode yang Anda gunakan bergantung pada data Anda. Kita akan mencoba K-Means karena ini adalah teknik klasterisasi yang paling umum. Mari kita mulai!

Istilah yang akan Anda pelajari:

-   Skor Silhouette

-   Metode Elbow

-   Inersia

-   Variansi

### **Pendahuluan**

[Klasterisasi K-Means](https://wikipedia.org/wiki/K-means_clustering) adalah metode yang berasal dari domain pemrosesan sinyal. Metode ini digunakan untuk membagi dan mengelompokkan data ke dalam `k klaster` berdasarkan kesamaan fitur mereka.

Klaster dapat divisualisasikan sebagai [diagram Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), yang mencakup sebuah titik (atau 'seed') dan wilayah yang sesuai.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografis oleh Jen Looper</figcaption>

Klasterisasi K-Means memiliki langkah-langkah berikut:

1.  Ilmuwan data memulai dengan menentukan jumlah klaster yang diinginkan untuk dibuat.

2.  Selanjutnya, algoritma secara acak memilih K observasi dari dataset untuk dijadikan pusat awal klaster (yaitu, centroid).

3.  Kemudian, setiap observasi yang tersisa diberikan ke centroid terdekatnya.

4.  Selanjutnya, rata-rata baru dari setiap klaster dihitung dan centroid dipindahkan ke rata-rata tersebut.

5.  Setelah pusat-pusat diperbarui, setiap observasi diperiksa lagi untuk melihat apakah mungkin lebih dekat ke klaster yang berbeda. Semua objek ditugaskan ulang menggunakan rata-rata klaster yang diperbarui. Langkah penugasan klaster dan pembaruan centroid diulang secara iteratif hingga penugasan klaster berhenti berubah (yaitu, ketika konvergensi tercapai). Biasanya, algoritma berhenti ketika setiap iterasi baru menghasilkan pergerakan centroid yang sangat kecil dan klaster menjadi statis.

<div>

> Perlu dicatat bahwa karena proses pengacakan observasi awal k yang digunakan sebagai centroid awal, kita bisa mendapatkan hasil yang sedikit berbeda setiap kali kita menerapkan prosedur ini. Oleh karena itu, sebagian besar algoritma menggunakan beberapa *random starts* dan memilih iterasi dengan WCSS terendah. Karena itu, sangat disarankan untuk selalu menjalankan K-Means dengan beberapa nilai *nstart* untuk menghindari *optimum lokal yang tidak diinginkan.*

</div>

Animasi pendek ini menggunakan [karya seni](https://github.com/allisonhorst/stats-illustrations) dari Allison Horst menjelaskan proses klasterisasi:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>

Pertanyaan mendasar yang muncul dalam klasterisasi adalah: bagaimana Anda tahu berapa banyak klaster yang harus digunakan untuk memisahkan data Anda? Salah satu kelemahan menggunakan K-Means adalah Anda perlu menentukan `k`, yaitu jumlah `centroid`. Untungnya, `metode elbow` membantu memperkirakan nilai awal yang baik untuk `k`. Anda akan mencobanya sebentar lagi.

### 

**Prasyarat**

Kita akan melanjutkan dari tempat kita berhenti di [pelajaran sebelumnya](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), di mana kita menganalisis dataset, membuat banyak visualisasi, dan memfilter dataset ke observasi yang menarik. Pastikan untuk memeriksanya!

Kita memerlukan beberapa paket untuk menyelesaikan modul ini. Anda dapat menginstalnya dengan: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Sebagai alternatif, skrip di bawah ini akan memeriksa apakah Anda memiliki paket yang diperlukan untuk menyelesaikan modul ini dan menginstalnya untuk Anda jika ada yang belum terpasang.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Mari kita mulai!

## 1. Menari dengan data: Persempit ke 3 genre musik paling populer

Ini adalah rangkuman dari apa yang telah kita lakukan di pelajaran sebelumnya. Mari kita olah dan analisis beberapa data!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Itu berjalan dengan baik!

## 2. Eksplorasi data lebih lanjut.

Seberapa bersih data ini? Mari kita periksa outlier menggunakan boxplot. Kita akan fokus pada kolom numerik dengan sedikit outlier (meskipun Anda bisa membersihkan outlier tersebut). Boxplot dapat menunjukkan rentang data dan membantu memilih kolom mana yang akan digunakan. Perlu dicatat, boxplot tidak menunjukkan variansi, elemen penting dari data yang baik untuk pengelompokan. Silakan lihat [diskusi ini](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) untuk bacaan lebih lanjut.

[Boxplot](https://en.wikipedia.org/wiki/Box_plot) digunakan untuk menggambarkan distribusi data `numerik` secara grafis, jadi mari kita mulai dengan *memilih* semua kolom numerik bersama dengan genre musik populer.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Lihat bagaimana pembantu seleksi `where` membuat ini menjadi mudah 💁? Jelajahi fungsi-fungsi lainnya [di sini](https://tidyselect.r-lib.org/).

Karena kita akan membuat boxplot untuk setiap fitur numerik dan ingin menghindari penggunaan loop, mari kita ubah format data kita menjadi format *lebih panjang* yang memungkinkan kita memanfaatkan `facets` - subplot yang masing-masing menampilkan satu subset data.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Lebih panjang lagi! Sekarang saatnya untuk beberapa `ggplots`! Jadi, `geom` apa yang akan kita gunakan?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Easy-gg!

Sekarang kita bisa melihat bahwa data ini agak berisik: dengan mengamati setiap kolom sebagai boxplot, kamu bisa melihat adanya outlier. Kamu bisa menelusuri dataset dan menghapus outlier tersebut, tetapi itu akan membuat data menjadi sangat minimal.

Untuk saat ini, mari kita pilih kolom mana yang akan kita gunakan untuk latihan clustering kita. Mari pilih kolom numerik dengan rentang yang serupa. Kita bisa mengubah `artist_top_genre` menjadi numerik, tetapi untuk sekarang kita akan mengabaikannya.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Menghitung klasterisasi k-means di R

Kita dapat menghitung k-means di R menggunakan fungsi bawaan `kmeans`, lihat `help("kmeans()")`. Fungsi `kmeans()` menerima data frame dengan semua kolom numerik sebagai argumen utamanya.

Langkah pertama saat menggunakan klasterisasi k-means adalah menentukan jumlah klaster (k) yang akan dihasilkan dalam solusi akhir. Kita tahu ada 3 genre lagu yang telah kita pisahkan dari dataset, jadi mari kita coba 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Objek kmeans berisi beberapa informasi yang dijelaskan dengan baik di `help("kmeans()")`. Untuk saat ini, mari kita fokus pada beberapa hal. Kita melihat bahwa data telah dikelompokkan menjadi 3 klaster dengan ukuran 65, 110, 111. Outputnya juga mencakup pusat klaster (rata-rata) untuk 3 kelompok tersebut di 5 variabel.

Vektor klaster adalah penugasan klaster untuk setiap observasi. Mari kita gunakan fungsi `augment` untuk menambahkan penugasan klaster ke dalam set data asli.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Sempurna, kita baru saja membagi data kita menjadi 3 kelompok. Jadi, seberapa baik pengelompokan kita 🤷? Mari kita lihat `Silhouette score`.

### **Silhouette score**

[Analisis Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) dapat digunakan untuk mempelajari jarak pemisahan antara kelompok-kelompok yang dihasilkan. Skor ini berkisar dari -1 hingga 1, dan jika skor mendekati 1, kelompok tersebut padat dan terpisah dengan baik dari kelompok lainnya. Nilai yang mendekati 0 menunjukkan kelompok yang saling tumpang tindih dengan sampel yang sangat dekat dengan batas keputusan kelompok tetangga. [sumber](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metode rata-rata silhouette menghitung rata-rata silhouette dari pengamatan untuk berbagai nilai *k*. Skor rata-rata silhouette yang tinggi menunjukkan pengelompokan yang baik.

Fungsi `silhouette` dalam paket cluster digunakan untuk menghitung rata-rata lebar silhouette.

> Silhouette dapat dihitung dengan menggunakan [metrik jarak](https://en.wikipedia.org/wiki/Distance "Distance") apa pun, seperti [jarak Euclidean](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") atau [jarak Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") yang telah kita bahas di [pelajaran sebelumnya](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Skor kita adalah **.549**, jadi berada tepat di tengah. Ini menunjukkan bahwa data kita tidak terlalu cocok untuk jenis pengelompokan ini. Mari kita lihat apakah kita dapat mengonfirmasi dugaan ini secara visual. Paket [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) menyediakan fungsi (`fviz_cluster()`) untuk memvisualisasikan pengelompokan.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Tumpang tindih dalam kluster menunjukkan bahwa data kita tidak terlalu cocok untuk jenis pengelompokan ini, tetapi mari kita lanjutkan.

## 4. Menentukan jumlah kluster yang optimal

Pertanyaan mendasar yang sering muncul dalam pengelompokan K-Means adalah ini - tanpa label kelas yang diketahui, bagaimana kita tahu berapa banyak kluster yang harus digunakan untuk memisahkan data kita?

Salah satu cara untuk mencoba mengetahuinya adalah dengan menggunakan sampel data untuk `membuat serangkaian model pengelompokan` dengan jumlah kluster yang meningkat (misalnya dari 1-10), dan mengevaluasi metrik pengelompokan seperti **Silhouette score.**

Mari kita tentukan jumlah kluster yang optimal dengan menghitung algoritma pengelompokan untuk berbagai nilai *k* dan mengevaluasi **Within Cluster Sum of Squares** (WCSS). Total within-cluster sum of square (WCSS) mengukur kepadatan pengelompokan, dan kita ingin nilainya sekecil mungkin, dengan nilai yang lebih rendah menunjukkan bahwa titik data lebih dekat satu sama lain.

Mari kita eksplorasi efek dari berbagai pilihan `k`, dari 1 hingga 10, pada pengelompokan ini.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Sekarang setelah kita memiliki total jumlah kuadrat dalam kluster (tot.withinss) untuk setiap algoritma pengelompokan dengan pusat *k*, kita menggunakan [metode siku](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) untuk menemukan jumlah kluster yang optimal. Metode ini terdiri dari membuat plot WCSS sebagai fungsi dari jumlah kluster, dan memilih [siku kurva](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Siku kurva") sebagai jumlah kluster yang akan digunakan.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Plot menunjukkan penurunan besar dalam WCSS (jadi *kekompakan* yang lebih besar) saat jumlah klaster meningkat dari satu ke dua, dan penurunan yang cukup terlihat lagi dari dua ke tiga klaster. Setelah itu, penurunan menjadi kurang signifikan, menghasilkan sebuah `elbow` 💪 pada grafik di sekitar tiga klaster. Ini adalah indikasi yang baik bahwa terdapat dua hingga tiga klaster data yang cukup terpisah dengan baik.

Sekarang kita dapat melanjutkan dan mengekstrak model klastering di mana `k = 3`:

> `pull()`: digunakan untuk mengekstrak satu kolom
>
> `pluck()`: digunakan untuk mengindeks struktur data seperti daftar


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Bagus! Mari kita visualisasikan klaster yang telah diperoleh. Tertarik dengan interaktivitas menggunakan `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Mungkin kita akan mengharapkan bahwa setiap kluster (diwakili oleh warna yang berbeda) memiliki genre yang berbeda (diwakili oleh bentuk yang berbeda).

Mari kita lihat akurasi model.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Akurasi model ini tidak buruk, tetapi juga tidak terlalu baik. Mungkin data ini kurang cocok untuk K-Means Clustering. Data ini terlalu tidak seimbang, kurang berkorelasi, dan terdapat terlalu banyak variasi antar nilai kolom sehingga sulit untuk dikelompokkan dengan baik. Faktanya, klaster yang terbentuk kemungkinan besar sangat dipengaruhi atau terdistorsi oleh tiga kategori genre yang telah kita definisikan sebelumnya.

Namun demikian, ini adalah proses pembelajaran yang cukup menarik!

Dalam dokumentasi Scikit-learn, Anda dapat melihat bahwa model seperti ini, dengan klaster yang tidak terlalu jelas, memiliki masalah 'varian':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografis dari Scikit-learn</figcaption>



## **Varian**

Varian didefinisikan sebagai "rata-rata dari kuadrat perbedaan terhadap Mean" [sumber](https://www.mathsisfun.com/data/standard-deviation.html). Dalam konteks masalah clustering ini, varian mengacu pada data di mana angka-angka dalam dataset kita cenderung menyimpang terlalu jauh dari rata-rata.

✅ Ini adalah momen yang tepat untuk memikirkan semua cara yang dapat Anda lakukan untuk mengatasi masalah ini. Modifikasi data sedikit lebih banyak? Gunakan kolom yang berbeda? Gunakan algoritma yang berbeda? Petunjuk: Cobalah [menyesuaikan skala data Anda](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) untuk menormalkannya dan uji kolom lainnya.

> Cobalah '[kalkulator varian](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' ini untuk memahami konsepnya lebih dalam.

------------------------------------------------------------------------

## **🚀Tantangan**

Luangkan waktu dengan notebook ini, modifikasi parameter. Bisakah Anda meningkatkan akurasi model dengan membersihkan data lebih lanjut (misalnya, menghapus outlier)? Anda dapat menggunakan bobot untuk memberikan lebih banyak pengaruh pada sampel data tertentu. Apa lagi yang dapat Anda lakukan untuk menciptakan klaster yang lebih baik?

Petunjuk: Cobalah untuk menyesuaikan skala data Anda. Ada kode yang dikomentari di notebook yang menambahkan skala standar untuk membuat kolom data lebih mirip satu sama lain dalam hal rentang. Anda akan menemukan bahwa meskipun skor siluet menurun, 'tikungan' pada grafik siku menjadi lebih halus. Hal ini terjadi karena membiarkan data tidak disesuaikan skala memungkinkan data dengan varian lebih kecil memiliki pengaruh lebih besar. Baca lebih lanjut tentang masalah ini [di sini](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kuis setelah kuliah**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Tinjauan & Studi Mandiri**

-   Lihatlah Simulator K-Means [seperti ini](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Anda dapat menggunakan alat ini untuk memvisualisasikan titik data sampel dan menentukan centroid-nya. Anda dapat mengedit tingkat keacakan data, jumlah klaster, dan jumlah centroid. Apakah ini membantu Anda memahami bagaimana data dapat dikelompokkan?

-   Selain itu, lihat [handout tentang K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) dari Stanford.

Ingin mencoba keterampilan clustering yang baru Anda pelajari pada dataset yang cocok untuk K-Means clustering? Silakan lihat:

-   [Latih dan Evaluasi Model Clustering](https://rpubs.com/eR_ic/clustering) menggunakan Tidymodels dan teman-temannya

-   [Analisis Klaster K-Means](https://uc-r.github.io/kmeans_clustering), Panduan Pemrograman R UC Business Analytics

- [K-Means clustering dengan prinsip data yang rapi](https://www.tidymodels.org/learn/statistics/k-means/)

## **Tugas**

[Cobalah metode clustering yang berbeda](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## TERIMA KASIH KEPADA:

[Jen Looper](https://www.twitter.com/jenlooper) atas pembuatan versi Python asli dari modul ini ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) atas ilustrasi luar biasa yang membuat R lebih ramah dan menarik. Temukan lebih banyak ilustrasi di [galerinya](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Selamat Belajar,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
